# 🔋 EV Battery Digital Twin - Model Training Notebook

This notebook trains two ML models using your **EV Predictive Maintenance Dataset**:
1. **RUL Model**: Predicts Remaining Useful Life (cycles remaining)
2. **Failure Probability Model**: Predicts probability of battery failure

## 📋 Instructions
1. Run all cells in order (Ctrl+Shift+Enter)
2. Models will be saved in the `models/` folder
3. Use these models with your live predictor

## 📊 Dataset Info
- **File**: `EV_Predictive_Maintenance_Dataset_15min.csv`
- **Records**: 175,393 samples
- **Features**: 30 columns (battery, motor, environmental data)
- **Targets**: RUL (Remaining Useful Life) and Failure_Probability

## 📦 Step 1: Install Required Packages

In [ ]:
# Install required packages
import subprocess
import sys

packages = ['numpy', 'pandas', 'scikit-learn', 'xgboost', 'matplotlib', 'seaborn', 'joblib']

for package in packages:
    try:
        __import__(package if package != 'scikit-learn' else 'sklearn')
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

print("✅ All packages ready!")

## 📚 Step 2: Import Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, classification_report
import xgboost as xgb
import joblib
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("✅ All libraries imported successfully!")
print(f"XGBoost version: {xgb.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

## 📁 Step 3: Load Dataset

Loading the EV Predictive Maintenance Dataset from the datasets folder.

In [ ]:
# Define path to dataset
# Try multiple possible paths
possible_paths = [
    '../datasets/EV_Predictive_Maintenance_Dataset_15min.csv',  # Running from notebooks folder
    'datasets/EV_Predictive_Maintenance_Dataset_15min.csv',     # Running from project root
    'EV_Predictive_Maintenance_Dataset_15min.csv',             # Running from datasets folder or uploaded
]

dataset_path = None
for path in possible_paths:
    if os.path.exists(path):
        dataset_path = path
        break

if dataset_path is None:
    print("❌ Dataset not found! Please ensure the file exists in one of these locations:")
    for path in possible_paths:
        print(f"   - {path}")
    print("\n💡 If using Google Colab, upload the file using:")
    print("   from google.colab import files")
    print("   uploaded = files.upload()")
    raise FileNotFoundError("Dataset not found")
    
print(f"📂 Loading dataset from: {dataset_path}")

# Load the dataset
df = pd.read_csv(dataset_path)

print(f"\n✅ Dataset loaded successfully!")
print(f"📊 Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"\n📋 Columns:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2d}. {col}")


## 🔍 Step 4: Data Exploration

In [ ]:
# Display first few rows
print("📊 First 5 rows of the dataset:")
display(df.head())

# Display last few rows
print("\n📊 Last 5 rows of the dataset:")
display(df.tail())

# Data types
print("\n📋 Data Types:")
print(df.dtypes)

In [ ]:
# Statistical summary
print("📈 Statistical Summary:")
display(df.describe())

# Check for missing values
print("\n🔍 Missing Values Check:")
missing = df.isnull().sum()
if missing.sum() == 0:
    print("✅ No missing values found!")
else:
    print("⚠️ Missing values detected:")
    print(missing[missing > 0])

# Check for duplicates
duplicates = df.duplicated().sum()
print(f"\n🔍 Duplicate rows: {duplicates}")

In [ ]:
# Check target variables
print("🎯 Target Variable Analysis:")
print("\n1. RUL (Remaining Useful Life):")
print(f"   Min: {df['RUL'].min():.2f} cycles")
print(f"   Max: {df['RUL'].max():.2f} cycles")
print(f"   Mean: {df['RUL'].mean():.2f} cycles")
print(f"   Median: {df['RUL'].median():.2f} cycles")

print("\n2. Failure_Probability:")
print(f"   Min: {df['Failure_Probability'].min()}")
print(f"   Max: {df['Failure_Probability'].max()}")
print(f"   Mean: {df['Failure_Probability'].mean():.4f}")
print(f"   Unique values: {df['Failure_Probability'].nunique()}")
print(f"   Value distribution:\n{df['Failure_Probability'].value_counts().sort_index()}")

## 📊 Step 5: Data Visualization

In [ ]:
# Plot target distributions
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# RUL distribution
axes[0].hist(df['RUL'], bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[0].set_title('RUL Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Remaining Useful Life (cycles)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].grid(True, alpha=0.3)
axes[0].axvline(df['RUL'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df["RUL"].mean():.1f}')
axes[0].legend()

# Failure Probability distribution
failure_counts = df['Failure_Probability'].value_counts().sort_index()
axes[1].bar(failure_counts.index, failure_counts.values, color='salmon', edgecolor='black', alpha=0.7)
axes[1].set_title('Failure Probability Distribution', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Failure Probability', fontsize=12)
axes[1].set_ylabel('Count', fontsize=12)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# Key battery features visualization
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
fig.suptitle('Key Battery Features Distribution', fontsize=16, fontweight='bold')

features_to_plot = ['SoC', 'SoH', 'Battery_Voltage', 'Battery_Current', 'Battery_Temperature', 'Charge_Cycles']

for idx, feature in enumerate(features_to_plot):
    ax = axes[idx // 3, idx % 3]
    ax.hist(df[feature], bins=50, color='lightgreen', edgecolor='black', alpha=0.7)
    ax.set_title(feature, fontweight='bold', fontsize=12)
    ax.set_xlabel('Value', fontsize=10)
    ax.set_ylabel('Frequency', fontsize=10)
    ax.grid(True, alpha=0.3)
    ax.axvline(df[feature].mean(), color='red', linestyle='--', linewidth=2, alpha=0.7)

plt.tight_layout()
plt.show()

In [ ]:
# Correlation analysis for battery features
battery_features = ['SoC', 'SoH', 'Battery_Voltage', 'Battery_Current', 
                   'Battery_Temperature', 'Charge_Cycles', 'RUL', 'Failure_Probability']

plt.figure(figsize=(10, 8))
correlation = df[battery_features].corr()
sns.heatmap(correlation, annot=True, fmt='.2f', cmap='RdYlGn', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Battery Features Correlation Heatmap', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n🔍 Strong correlations with RUL:")
rul_corr = correlation['RUL'].sort_values(ascending=False)
print(rul_corr[rul_corr.abs() > 0.3])

## 🔧 Step 6: Feature Engineering & Selection

We'll select the most relevant features for prediction based on domain knowledge and correlation analysis.

In [ ]:
# Select features for modeling
# ⚠️ IMPORTANT: Using only COMPLETE features (removed 5 features with ~94% missing values)
# Removed: Driving_Speed, Ambient_Temperature, Load_Weight, Distance_Traveled, Component_Health_Score
# This will retain 170,000+ samples instead of only 9,899 samples!

selected_features = [
    'SoC',                    # State of Charge (matches your system) - COMPLETE
    'SoH',                    # State of Health (matches your system) - COMPLETE
    'Battery_Voltage',        # Battery Voltage (matches your system) - COMPLETE
    'Battery_Current',        # Battery Current (matches your system) - COMPLETE
    'Battery_Temperature',    # Battery Temperature (matches your system) - COMPLETE
    'Charge_Cycles',          # Battery cycles (important for RUL) - COMPLETE
    'Power_Consumption',      # Energy usage pattern - COMPLETE
]

print("✅ Selected features for modeling (COMPLETE FEATURES ONLY):")
print(f"   Using {len(selected_features)} features with no missing values")
print(f"   Expected to retain >97% of original data (~170,000 samples)\n")

for i, feat in enumerate(selected_features, 1):
    print(f"  {i}. {feat}")

# Verify all features exist
missing_features = [f for f in selected_features if f not in df.columns]
if missing_features:
    print(f"\n⚠️ Warning: Missing features: {missing_features}")
    selected_features = [f for f in selected_features if f in df.columns]

# Prepare feature matrix
X = df[selected_features].copy()

# Prepare targets
y_rul = df['RUL'].copy()
y_failure = df['Failure_Probability'].copy()

print(f"\n📊 Data prepared:")
print(f"   Features (X): {X.shape}")
print(f"   RUL target (y): {y_rul.shape}")
print(f"   Failure target (y): {y_failure.shape}")

# Check missing values in selected features
missing_in_selected = X.isnull().sum()
if missing_in_selected.sum() > 0:
    print(f"\n⚠️ Missing values in selected features:")
    for feat, count in missing_in_selected[missing_in_selected > 0].items():
        print(f"   {feat}: {count:,} ({count/len(X)*100:.2f}%)")
else:
    print(f"\n✅ All selected features are complete (no missing values)!")


In [ ]:
# Data cleaning: Handle missing and infinite values
print("\n🔍 Checking for invalid values...")
print(f"   X NaN count: {X.isnull().sum().sum()}")
print(f"   X Inf count: {np.isinf(X).sum().sum()}")
print(f"   y_rul NaN count: {y_rul.isnull().sum()}")
print(f"   y_rul Inf count: {np.isinf(y_rul).sum()}")
print(f"   y_failure NaN count: {y_failure.isnull().sum()}")
print(f"   y_failure Inf count: {np.isinf(y_failure).sum()}")

# Remove rows with NaN or Inf in features or targets
initial_size = len(X)
valid_indices = ~(X.isnull().any(axis=1) | 
                  np.isinf(X).any(axis=1) | 
                  y_rul.isnull() | 
                  np.isinf(y_rul) |
                  y_failure.isnull() |
                  np.isinf(y_failure))

X = X[valid_indices]
y_rul = y_rul[valid_indices]
y_failure = y_failure[valid_indices]

removed_count = initial_size - len(X)
if removed_count > 0:
    print(f"\n⚠️ Removed {removed_count:,} rows with invalid values ({removed_count/initial_size*100:.2f}%)")
    print(f"✅ Clean dataset: {len(X):,} samples remaining")
else:
    print(f"\n✅ No invalid values found - dataset is clean!")

print(f"\n📊 Final data shape:")
print(f"   Features (X): {X.shape}")
print(f"   RUL target: {y_rul.shape}")
print(f"   Failure target: {y_failure.shape}")


## 🔍 Data Quality Analysis

Let's examine why we have so few samples and check for data distribution issues.

In [ ]:
# Analyze why we have so few clean samples
print("📊 Data Loss Analysis:")
print("="*60)
print(f"Original dataset size: {len(df):,} samples")
print(f"After feature selection: {initial_size:,} samples")
print(f"After cleaning: {len(X):,} samples")
print(f"Data loss: {initial_size - len(X):,} samples ({(initial_size - len(X))/initial_size*100:.2f}%)")

print("\n🔍 Analyzing which features caused the most data loss...")
feature_nan_counts = {}
for feat in selected_features:
    nan_count = df[feat].isnull().sum()
    if nan_count > 0:
        feature_nan_counts[feat] = nan_count

if feature_nan_counts:
    print("\nFeatures with missing values:")
    for feat, count in sorted(feature_nan_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"   {feat:<25s}: {count:>8,} ({count/len(df)*100:>5.2f}%)")
else:
    print("   No missing values in selected features")

# Check target distribution
print("\n📊 Target Variable Statistics (Clean Data):")
print(f"RUL range: [{y_rul.min():.2f}, {y_rul.max():.2f}] cycles")
print(f"RUL mean: {y_rul.mean():.2f} ± {y_rul.std():.2f} cycles")
print(f"Failure distribution: {y_failure.value_counts().to_dict()}")

# Warning if dataset is too small
if len(X) < 5000:
    print("\n⚠️ WARNING: Dataset is very small after cleaning!")
    print(f"   Current size: {len(X):,} samples")
    print(f"   Recommended: >10,000 samples for robust training")
    print(f"   This may lead to overfitting or poor generalization")


## 🔀 Step 7: Train-Test Split

In [ ]:
# Split data (80% train, 20% test)
X_train, X_test, y_rul_train, y_rul_test = train_test_split(
    X, y_rul, test_size=0.2, random_state=42, shuffle=True
)

_, _, y_failure_train, y_failure_test = train_test_split(
    X, y_failure, test_size=0.2, random_state=42, shuffle=True
)

print("✅ Data split completed!")
print(f"\n📊 Training set: {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"📊 Test set: {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X)*100:.1f}%)")

# Check for invalid values before scaling
print(f"\n🔍 Data Quality Check:")
x_train_nan = X_train.isnull().sum().sum()
x_train_inf = np.isinf(X_train.select_dtypes(include=[np.number])).sum().sum()
y_rul_train_nan = y_rul_train.isnull().sum()
y_rul_train_inf = np.isinf(y_rul_train).sum()
y_failure_train_nan = y_failure_train.isnull().sum()
y_failure_train_inf = np.isinf(y_failure_train).sum()

print(f"   X_train NaN: {x_train_nan}")
print(f"   X_train Inf: {x_train_inf}")
print(f"   y_rul_train NaN: {y_rul_train_nan}")
print(f"   y_rul_train Inf: {y_rul_train_inf}")
print(f"   y_failure_train NaN: {y_failure_train_nan}")
print(f"   y_failure_train Inf: {y_failure_train_inf}")

# Clean data: remove NaN and Inf values
total_invalid_train = x_train_nan + x_train_inf + y_rul_train_nan + y_rul_train_inf + y_failure_train_nan + y_failure_train_inf

if total_invalid_train > 0:
    print(f"\n⚠️ Cleaning {total_invalid_train} invalid values from training data...")
    
    # Create mask for valid rows (convert DataFrame operations to boolean array)
    valid_mask_train = ~(X_train.isnull().any(axis=1).values | 
                         np.isinf(X_train.select_dtypes(include=[np.number])).any(axis=1).values | 
                         y_rul_train.isnull().values | 
                         np.isinf(y_rul_train).values |
                         y_failure_train.isnull().values |
                         np.isinf(y_failure_train).values)
    
    X_train = X_train[valid_mask_train]
    y_rul_train = y_rul_train[valid_mask_train]
    y_failure_train = y_failure_train[valid_mask_train]
    
    print(f"   Removed {(~valid_mask_train).sum():,} invalid training samples")
    print(f"   Training set now: {X_train.shape[0]:,} samples")

# Check test set
x_test_nan = X_test.isnull().sum().sum()
x_test_inf = np.isinf(X_test.select_dtypes(include=[np.number])).sum().sum()
y_rul_test_nan = y_rul_test.isnull().sum()
y_rul_test_inf = np.isinf(y_rul_test).sum()
y_failure_test_nan = y_failure_test.isnull().sum()
y_failure_test_inf = np.isinf(y_failure_test).sum()

total_invalid_test = x_test_nan + x_test_inf + y_rul_test_nan + y_rul_test_inf + y_failure_test_nan + y_failure_test_inf

if total_invalid_test > 0:
    print(f"\n⚠️ Cleaning {total_invalid_test} invalid values from test data...")
    
    # Create mask for valid rows (convert DataFrame operations to boolean array)
    valid_mask_test = ~(X_test.isnull().any(axis=1).values | 
                        np.isinf(X_test.select_dtypes(include=[np.number])).any(axis=1).values | 
                        y_rul_test.isnull().values | 
                        np.isinf(y_rul_test).values |
                        y_failure_test.isnull().values |
                        np.isinf(y_failure_test).values)
    
    X_test = X_test[valid_mask_test]
    y_rul_test = y_rul_test[valid_mask_test]
    y_failure_test = y_failure_test[valid_mask_test]
    
    print(f"   Removed {(~valid_mask_test).sum():,} invalid test samples")
    print(f"   Test set now: {X_test.shape[0]:,} samples")

print(f"\n✅ Data cleaning completed!")
print(f"   Final training set: {X_train.shape[0]:,} samples")
print(f"   Final test set: {X_test.shape[0]:,} samples")

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\n✅ Feature scaling completed")
print(f"   Scaled mean: {X_train_scaled.mean():.6f} (should be ~0)")
print(f"   Scaled std: {X_train_scaled.std():.6f} (should be ~1)")


## 🤖 Step 8: Train RUL Prediction Model

Training XGBoost model to predict Remaining Useful Life.

In [ ]:
print("🔄 Training RUL Model (XGBoost Regressor)...")
print("="*60)

# Configure XGBoost for RUL prediction with reduced overfitting
rul_model = xgb.XGBRegressor(
    n_estimators=100,           # Reduced from 300 to prevent overfitting
    max_depth=4,                # Reduced from 8 to prevent overfitting
    learning_rate=0.1,          # Increased from 0.05 for better convergence
    subsample=0.7,              # More aggressive subsampling
    colsample_bytree=0.7,       # More aggressive feature sampling
    objective='reg:squarederror',
    random_state=42,
    n_jobs=-1,
    min_child_weight=5,         # Increased from 3 to prevent overfitting
    gamma=0.2,                  # Increased regularization
    reg_alpha=0.1,              # L1 regularization
    reg_lambda=1.0,             # L2 regularization
    early_stopping_rounds=10    # Stop if no improvement
)

print("Training with cross-validation and early stopping...")
print("This may take a few moments...\n")

# Train the model with early stopping
rul_model.fit(
    X_train_scaled, y_rul_train,
    eval_set=[(X_train_scaled, y_rul_train), (X_test_scaled, y_rul_test)],
    verbose=False
)

print("✅ RUL Model training completed!")
print(f"   Best iteration: {rul_model.best_iteration}")
print(f"   Total estimators used: {rul_model.n_estimators}")


In [ ]:
# Evaluate RUL model
y_rul_pred_train = rul_model.predict(X_train_scaled)
y_rul_pred_test = rul_model.predict(X_test_scaled)

# Training metrics
train_mae = mean_absolute_error(y_rul_train, y_rul_pred_train)
train_rmse = np.sqrt(mean_squared_error(y_rul_train, y_rul_pred_train))
train_r2 = r2_score(y_rul_train, y_rul_pred_train)

# Test metrics
test_mae = mean_absolute_error(y_rul_test, y_rul_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_rul_test, y_rul_pred_test))
test_r2 = r2_score(y_rul_test, y_rul_pred_test)

print("\n📊 RUL Model Performance:")
print("="*60)
print(f"{'Metric':<25} {'Training':<15} {'Test':<15}")
print("-"*60)
print(f"{'MAE (cycles)':<25} {train_mae:>14.2f} {test_mae:>14.2f}")
print(f"{'RMSE (cycles)':<25} {train_rmse:>14.2f} {test_rmse:>14.2f}")
print(f"{'R² Score':<25} {train_r2:>14.4f} {test_r2:>14.4f}")
print("="*60)

# Detailed diagnosis
if test_r2 < 0:
    print("🚨 CRITICAL: Negative R² Score Detected!")
    print("   This means the model performs worse than a simple mean baseline.")
    print("   Possible causes:")
    print("   1. Severe overfitting (model memorized training noise)")
    print("   2. Data distribution mismatch between train/test")
    print("   3. Too few samples for reliable training")
    print("   4. Features have different distributions in train/test")
    print("\n   Current dataset size: {} samples".format(len(X)))
    print("   Train/Test split may have created distribution mismatch")
elif test_r2 > 0.9:
    print("✅ Excellent model performance! (R² > 0.9)")
elif test_r2 > 0.8:
    print("✅ Good model performance! (R² > 0.8)")
elif test_r2 > 0.7:
    print("✅ Acceptable performance (R² > 0.7)")
elif test_r2 > 0.5:
    print("⚠️ Moderate performance (R² > 0.5)")
    print("   Model has some predictive power but could be improved")
else:
    print("⚠️ Poor performance (R² < 0.5)")
    print("   Model needs significant improvement")

# Calculate baseline comparison
baseline_pred = np.full_like(y_rul_test, y_rul_train.mean())
baseline_rmse = np.sqrt(mean_squared_error(y_rul_test, baseline_pred))
print(f"\n📊 Baseline (mean prediction) RMSE: {baseline_rmse:.2f} cycles")
print(f"   Model RMSE: {test_rmse:.2f} cycles")
if test_rmse < baseline_rmse:
    improvement = (baseline_rmse - test_rmse) / baseline_rmse * 100
    print(f"   ✅ Model is {improvement:.1f}% better than baseline")
else:
    degradation = (test_rmse - baseline_rmse) / baseline_rmse * 100
    print(f"   ❌ Model is {degradation:.1f}% worse than baseline")


In [ ]:
# Visualize RUL predictions
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. Actual vs Predicted (Test Set)
axes[0].scatter(y_rul_test, y_rul_pred_test, alpha=0.4, s=10, color='blue')
axes[0].plot([y_rul_test.min(), y_rul_test.max()], 
             [y_rul_test.min(), y_rul_test.max()], 
             'r--', lw=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual RUL (cycles)', fontsize=12)
axes[0].set_ylabel('Predicted RUL (cycles)', fontsize=12)
axes[0].set_title(f'RUL Predictions - Test Set (R²={test_r2:.4f})', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 2. Residuals Plot
residuals = y_rul_test - y_rul_pred_test
axes[1].scatter(y_rul_pred_test, residuals, alpha=0.4, s=10, color='green')
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Predicted RUL (cycles)', fontsize=12)
axes[1].set_ylabel('Residuals (cycles)', fontsize=12)
axes[1].set_title('Residual Plot', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# 3. Error Distribution
axes[2].hist(residuals, bins=50, color='orange', edgecolor='black', alpha=0.7)
axes[2].axvline(x=0, color='r', linestyle='--', lw=2, label='Zero Error')
axes[2].set_xlabel('Prediction Error (cycles)', fontsize=12)
axes[2].set_ylabel('Frequency', fontsize=12)
axes[2].set_title(f'Error Distribution (MAE={test_mae:.2f})', fontsize=13, fontweight='bold')
axes[2].legend()
axes[2].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# Feature importance for RUL model
importance = rul_model.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': selected_features,
    'Importance': importance
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(12, 6))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'], 
         color='skyblue', edgecolor='black')
plt.xlabel('Importance Score', fontsize=12)
plt.ylabel('Features', fontsize=12)
plt.title('RUL Model - Feature Importance', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print("\n🔍 Top 5 Most Important Features for RUL Prediction:")
for i, row in feature_importance_df.head(5).iterrows():
    print(f"  {row['Feature']:<25s}: {row['Importance']:.4f}")

## 🤖 Step 9: Train Failure Probability Model

Training XGBoost model to predict Failure Probability (binary classification).

In [ ]:
print("🔄 Training Failure Prediction Model (XGBoost Regressor)...")
print("="*60)

# Note: Failure_Probability appears to be binary (0 or 1)
# Configure with regularization to prevent overfitting

failure_model = xgb.XGBRegressor(
    n_estimators=100,           # Reduced from 300
    max_depth=3,                # Reduced from 6
    learning_rate=0.1,          # Increased from 0.05
    subsample=0.7,              # More aggressive subsampling
    colsample_bytree=0.7,       # More aggressive feature sampling
    objective='reg:squarederror',
    random_state=42,
    n_jobs=-1,
    min_child_weight=5,         # Increased from 2
    gamma=0.2,                  # Increased regularization
    reg_alpha=0.1,              # L1 regularization
    reg_lambda=1.0,             # L2 regularization
    early_stopping_rounds=10    # Stop if no improvement
)

print("Training with cross-validation and early stopping...")
print("This may take a few moments...\n")

# Train the model with early stopping
failure_model.fit(
    X_train_scaled, y_failure_train,
    eval_set=[(X_train_scaled, y_failure_train), (X_test_scaled, y_failure_test)],
    verbose=False
)

print("✅ Failure Model training completed!")
print(f"   Best iteration: {failure_model.best_iteration}")
print(f"   Total estimators used: {failure_model.n_estimators}")


In [ ]:
# Evaluate Failure model
y_failure_pred_train = failure_model.predict(X_train_scaled)
y_failure_pred_test = failure_model.predict(X_test_scaled)

# Clip predictions to [0, 1] range
y_failure_pred_train = np.clip(y_failure_pred_train, 0, 1)
y_failure_pred_test = np.clip(y_failure_pred_test, 0, 1)

# Metrics for regression
train_mae_f = mean_absolute_error(y_failure_train, y_failure_pred_train)
train_rmse_f = np.sqrt(mean_squared_error(y_failure_train, y_failure_pred_train))
train_r2_f = r2_score(y_failure_train, y_failure_pred_train)

test_mae_f = mean_absolute_error(y_failure_test, y_failure_pred_test)
test_rmse_f = np.sqrt(mean_squared_error(y_failure_test, y_failure_pred_test))
test_r2_f = r2_score(y_failure_test, y_failure_pred_test)

print("\n📊 Failure Model Performance:")
print("="*60)
print(f"{'Metric':<25} {'Training':<15} {'Test':<15}")
print("-"*60)
print(f"{'MAE':<25} {train_mae_f:>14.4f} {test_mae_f:>14.4f}")
print(f"{'RMSE':<25} {train_rmse_f:>14.4f} {test_rmse_f:>14.4f}")
print(f"{'R² Score':<25} {train_r2_f:>14.4f} {test_r2_f:>14.4f}")
print("="*60)

# If targets are binary, also show classification metrics
if set(y_failure_test.unique()).issubset({0, 1}):
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    
    y_failure_class = (y_failure_pred_test > 0.5).astype(int)
    
    accuracy = accuracy_score(y_failure_test, y_failure_class)
    precision = precision_score(y_failure_test, y_failure_class, zero_division=0)
    recall = recall_score(y_failure_test, y_failure_class, zero_division=0)
    f1 = f1_score(y_failure_test, y_failure_class, zero_division=0)
    
    print(f"\n📊 Classification Metrics (threshold=0.5):")
    print("-"*60)
    print(f"  Accuracy:  {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1 Score:  {f1:.4f}")

if test_r2_f > 0.8:
    print("\n✅ Excellent model performance!")
elif test_r2_f > 0.6:
    print("\n✅ Good model performance!")
else:
    print("\n⚠️ Model performance is acceptable for binary classification")

In [ ]:
# Visualize Failure predictions
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. Actual vs Predicted
axes[0].scatter(y_failure_test, y_failure_pred_test, alpha=0.4, s=10, color='red')
axes[0].plot([0, 1], [0, 1], 'r--', lw=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual Failure Probability', fontsize=12)
axes[0].set_ylabel('Predicted Failure Probability', fontsize=12)
axes[0].set_title(f'Failure Predictions (R²={test_r2_f:.4f})', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_xlim([-0.1, 1.1])
axes[0].set_ylim([-0.1, 1.1])

# 2. Prediction Distribution
axes[1].hist(y_failure_pred_test, bins=50, color='salmon', edgecolor='black', alpha=0.7)
axes[1].axvline(x=0.5, color='red', linestyle='--', lw=2, label='Threshold=0.5')
axes[1].set_xlabel('Predicted Failure Probability', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('Prediction Distribution', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

# 3. Confusion Matrix (if binary)
if set(y_failure_test.unique()).issubset({0, 1}):
    from sklearn.metrics import confusion_matrix
    y_failure_class = (y_failure_pred_test > 0.5).astype(int)
    cm = confusion_matrix(y_failure_test, y_failure_class)
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Reds', ax=axes[2], 
                xticklabels=['No Failure', 'Failure'],
                yticklabels=['No Failure', 'Failure'])
    axes[2].set_xlabel('Predicted', fontsize=12)
    axes[2].set_ylabel('Actual', fontsize=12)
    axes[2].set_title('Confusion Matrix', fontsize=13, fontweight='bold')
else:
    # Residuals if not binary
    residuals_f = y_failure_test - y_failure_pred_test
    axes[2].scatter(y_failure_pred_test, residuals_f, alpha=0.4, s=10, color='purple')
    axes[2].axhline(y=0, color='r', linestyle='--', lw=2)
    axes[2].set_xlabel('Predicted Failure Probability', fontsize=12)
    axes[2].set_ylabel('Residuals', fontsize=12)
    axes[2].set_title('Residual Plot', fontsize=13, fontweight='bold')
    axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Feature importance for Failure model
importance_f = failure_model.feature_importances_
feature_importance_df_f = pd.DataFrame({
    'Feature': selected_features,
    'Importance': importance_f
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(12, 6))
plt.barh(feature_importance_df_f['Feature'], feature_importance_df_f['Importance'], 
         color='salmon', edgecolor='black')
plt.xlabel('Importance Score', fontsize=12)
plt.ylabel('Features', fontsize=12)
plt.title('Failure Model - Feature Importance', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print("\n🔍 Top 5 Most Important Features for Failure Prediction:")
for i, row in feature_importance_df_f.head(5).iterrows():
    print(f"  {row['Feature']:<25s}: {row['Importance']:.4f}")

## 💾 Step 10: Save Models and Artifacts

In [ ]:
# Create models directory
models_dir = '../models'
if not os.path.exists(models_dir):
    models_dir = 'models'
    
os.makedirs(models_dir, exist_ok=True)

# Save RUL model
rul_model_path = os.path.join(models_dir, 'rul_xgb_model.joblib')
joblib.dump(rul_model, rul_model_path)
print(f"✅ RUL model saved: {rul_model_path}")

# Save Failure model
failure_model_path = os.path.join(models_dir, 'failure_xgb_model.joblib')
joblib.dump(failure_model, failure_model_path)
print(f"✅ Failure model saved: {failure_model_path}")

# Save scaler
scaler_path = os.path.join(models_dir, 'scaler.joblib')
joblib.dump(scaler, scaler_path)
print(f"✅ Scaler saved: {scaler_path}")

# Save feature names
feature_names_path = os.path.join(models_dir, 'feature_names.joblib')
joblib.dump(selected_features, feature_names_path)
print(f"✅ Feature names saved: {feature_names_path}")

# Save model metadata
metadata = {
    'training_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
    'n_samples_train': len(X_train),
    'n_samples_test': len(X_test),
    'n_features': len(selected_features),
    'features': selected_features,
    'rul_metrics': {
        'test_mae': float(test_mae),
        'test_rmse': float(test_rmse),
        'test_r2': float(test_r2)
    },
    'failure_metrics': {
        'test_mae': float(test_mae_f),
        'test_rmse': float(test_rmse_f),
        'test_r2': float(test_r2_f)
    }
}

metadata_path = os.path.join(models_dir, 'model_metadata.joblib')
joblib.dump(metadata, metadata_path)
print(f"✅ Metadata saved: {metadata_path}")

print("\n" + "="*60)
print("🎉 All models and artifacts saved successfully!")
print("="*60)

## 📊 Step 11: Final Summary Report

In [ ]:
print("\n" + "="*70)
print("📊 FINAL MODEL TRAINING SUMMARY")
print("="*70)

print(f"\n📁 Dataset Information:")
print("-"*70)
print(f"  Dataset: EV_Predictive_Maintenance_Dataset_15min.csv")
print(f"  Total Records: {len(df):,}")
print(f"  Training Samples: {len(X_train):,} ({len(X_train)/len(df)*100:.1f}%)")
print(f"  Test Samples: {len(X_test):,} ({len(X_test)/len(df)*100:.1f}%)")
print(f"  Features Used: {len(selected_features)}")

print(f"\n🔋 RUL Prediction Model:")
print("-"*70)
print(f"  Algorithm: XGBoost Regressor")
print(f"  Test MAE: {test_mae:.2f} cycles")
print(f"  Test RMSE: {test_rmse:.2f} cycles")
print(f"  Test R²: {test_r2:.4f}")
print(f"  Status: {'✅ Excellent' if test_r2 > 0.9 else '✅ Good' if test_r2 > 0.8 else '⚠️ Acceptable'}")

print(f"\n⚠️ Failure Prediction Model:")
print("-"*70)
print(f"  Algorithm: XGBoost Regressor")
print(f"  Test MAE: {test_mae_f:.4f}")
print(f"  Test RMSE: {test_rmse_f:.4f}")
print(f"  Test R²: {test_r2_f:.4f}")
print(f"  Status: {'✅ Excellent' if test_r2_f > 0.8 else '✅ Good' if test_r2_f > 0.6 else '⚠️ Acceptable'}")

print(f"\n💾 Saved Files:")
print("-"*70)
print(f"  1. {rul_model_path}")
print(f"  2. {failure_model_path}")
print(f"  3. {scaler_path}")
print(f"  4. {feature_names_path}")
print(f"  5. {metadata_path}")

print(f"\n🔍 Top 3 Important Features (RUL):")
for i, row in feature_importance_df.head(3).iterrows():
    print(f"  {row['Feature']:<25s}: {row['Importance']:.4f}")

print(f"\n🔍 Top 3 Important Features (Failure):")
for i, row in feature_importance_df_f.head(3).iterrows():
    print(f"  {row['Feature']:<25s}: {row['Importance']:.4f}")

print("\n" + "="*70)
print("✅ TRAINING COMPLETE! Models ready for deployment.")
print("="*70)

print("\n📝 Next Steps:")
print("  1. Models are saved in the 'models' folder")
print("  2. Update your predictor to use these trained models")
print("  3. Run: python src/inference/live_predictor.py")

## 🧪 Step 12: Test Prediction Function

Test the models with sample data to ensure they work correctly.

In [ ]:
# Create a sample input matching your live system format
print("🧪 Testing models with sample data...\n")

# Sample input (using median values from dataset)
sample_input = pd.DataFrame({
    'SoC': [0.85],
    'SoH': [0.94],
    'Battery_Voltage': [df['Battery_Voltage'].median()],
    'Battery_Current': [df['Battery_Current'].median()],
    'Battery_Temperature': [df['Battery_Temperature'].median()],
    'Driving_Speed': [60.0],
    'Charge_Cycles': [df['Charge_Cycles'].median()],
    'Power_Consumption': [df['Power_Consumption'].median()],
    'Ambient_Temperature': [25.0],
    'Load_Weight': [df['Load_Weight'].median()],
    'Distance_Traveled': [df['Distance_Traveled'].median()],
    'Component_Health_Score': [0.85]
})

print("📊 Sample Input:")
print("-"*60)
for col in sample_input.columns:
    print(f"  {col:<25s}: {sample_input[col].values[0]:>10.3f}")

# Scale the input
sample_scaled = scaler.transform(sample_input)

# Make predictions
rul_prediction = rul_model.predict(sample_scaled)[0]
failure_prediction = np.clip(failure_model.predict(sample_scaled)[0], 0, 1)

print("\n🔮 Model Predictions:")
print("-"*60)
print(f"  Remaining Useful Life:    {rul_prediction:>8.2f} cycles")
print(f"  Failure Probability:      {failure_prediction:>8.4f} ({failure_prediction*100:>6.2f}%)")

# Determine health status
if failure_prediction < 0.3:
    status = "✅ HEALTHY"
    color = "green"
elif failure_prediction < 0.6:
    status = "⚠️ WARNING"
    color = "orange"
else:
    status = "🚨 CRITICAL"
    color = "red"

print(f"\n  Battery Status:           {status}")
print("="*60)
print("✅ Models are working correctly!")

## 🎉 Training Complete!

Your models have been successfully trained and saved. You can now use them with your live predictor.

### 📦 Model Files Location
All files are in: `models/` folder

### 🚀 Next Steps
1. Ensure models are in your project's `models/` folder
2. Start your Docker services: `docker compose up -d`
3. Start the simulator: `python src/simulator/publisher.py`
4. Start the predictor: `python src/inference/live_predictor.py`
5. View dashboards at http://localhost:3000

### 📚 Model Information
- **RUL Model**: Predicts remaining battery cycles
- **Failure Model**: Predicts failure probability (0-1)
- **Features**: 12 carefully selected features
- **Algorithm**: XGBoost (tree-based ensemble)
- **Format**: Joblib (scikit-learn compatible)

**🎊 Happy Predicting!**